#Here we will examine the influence of seasonality on bacteria community composition at Lizard Island Research Station (LIRS) (E1 & E3)

In [1]:
cd ..

/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis


In [2]:
#Import libraries
from os.path import join,abspath
from os import listdir

In [3]:
#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map.txt")
output_dir = abspath("output/")

#Preference file indicating that Curtis-Brays distance should also be calculated by beta_diversity
bc_prefs_fp = abspath("input/bdiv_bc_prefs.txt")

In [9]:
#Filter only midshelf Lizard Island samples from E1 and E3. Sample IDs in: ids_E1_E3_Midshelf_Lizard_to_keep.txt
!filter_samples_from_otu_table.py -i $otu_table -o $output_dir/lizard_subset/otu_table_subset_lizard.biom -m $mapping --sample_id_fp $input_dir/ids_E1_E3_Midshelf_Lizard_to_keep.txt

In [10]:
#Summarize BIOM to confirm filter functioned correctly
!biom summarize-table -i $output_dir/lizard_subset/otu_table_subset_lizard.biom -o $output_dir/lizard_subset/otu_table_subset_lizard_summary.txt

In [12]:
#Run beta diversity on rarified (1000) subset of LIRS samples
lizard_otu_table_dir = join(output_dir,"lizard_subset")
depth = 1000

for c in listdir(lizard_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(output_dir,'bdiv_%s' %c)
    curr_otu_table = join(lizard_otu_table_dir,c)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.126159121179 and the largest is 11.4596209616.
  RuntimeWarning
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the

#Compare amongst compartments

In [34]:
#Export Sceleractinian coral compartment specific biom files (i.e. coral t, m and s)
lizard_otu_table = abspath("output/lizard_subset/otu_table_subset_lizard.biom")
subset_output = join(output_dir,'lizard_subset')
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $subset_output/lizard_subset_subset_mucus.biom -m $mapping -s 'sample_type:Coral_Mucus'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $subset_output/lizard_subset_subset_skeleton.biom -m $mapping -s 'sample_type:Coral_Skeleton'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $subset_output/lizard_subset_subset_tissue.biom -m $mapping -s 'sample_type:Coral_Tissue'

In [35]:
#Summarize BIOM to confirm filter functioned correctly
!biom summarize-table -i $subset_output/lizard_subset_subset_mucus.biom -o $subset_output/lizard_subset_subset_mucus_summary.txt
!biom summarize-table -i $subset_output/lizard_subset_subset_tissue.biom -o $subset_output/lizard_subset_subset_tissue_summary.txt
!biom summarize-table -i $subset_output/lizard_subset_subset_skeleton.biom -o $subset_output/lizard_subset_subset_skeleton_summary.txt

In [49]:
#Run beta diversity on rarified (1000) subset of each sample type
tissue_otu_table_dir = join(output_dir,"lizard_subset")
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/bdiv_lizard_subset_subset_mucus.biom'

If you need help with QIIME, see:
http://help.qiime.org
Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/bdiv_lizard_subset_subset_skeleton.biom'

If you need help with QIIME, see:
http://help.qiime.org
Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Proje

In [38]:
#Check for differences among mapping categories within a given compartment
lizard_otu_table_dir = join(output_dir,"lizard_subset")

n_permutations = 1000
categories = 'expedition_number,field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id'
for c in listdir(lizard_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(lizard_otu_table_dir,'%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(lizard_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

['.DS_Store', 'bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'lizard_subset_subset_mucus_even1000.biom', 'log_20160427165804.txt', 'unweighted_unifrac_dm.txt', 'unweighted_unifrac_emperor_pcoa_plot', 'unweighted_unifrac_pc.txt', 'weighted_unifrac_dm.txt', 'weighted_unifrac_emperor_pcoa_plot', 'weighted_unifrac_pc.txt']
curr_cat: expedition_number
curr_metric: bray_curtis
curr_category: bdiv_lizard_subset_subset_mucus.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                 Df SumsOfSqs MeanSqs F.Model      R2   Pr(>F)
qiime.data$map[[opts$category]]   1     3.700  3.7002  9.9494 0.08967 0.000999
Residuals                       101    37.562  0.3719         0.91033         
Total                           102    41.262                 1.00000         
      

OSError: [Errno 20] Not a directory: '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/lizard_subset_subset_mucus.biom'

In [41]:
#Boxplot summaries
n_permutations = 1000
categories = 'expedition_number,field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id,photosynthetically_active_radiation,visibility,turf_contact_percent,n_macroalgal_contacts,cyanobacteria_percent,cyanobacteria_contact,binary_turf_contact,binary_macroalgal_contact'
for c in listdir(lizard_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(lizard_otu_table_dir,'%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

['.DS_Store', 'bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'compare_categories_expedition_number_bray_curtis', 'compare_categories_expedition_number_unweighted_unifrac', 'compare_categories_expedition_number_weighted_unifrac', 'compare_categories_field_host_clade_sensu_fukami_bray_curtis', 'compare_categories_field_host_clade_sensu_fukami_unweighted_unifrac', 'compare_categories_field_host_clade_sensu_fukami_weighted_unifrac', 'compare_categories_field_host_genus_id_bray_curtis', 'compare_categories_field_host_genus_id_unweighted_unifrac', 'compare_categories_field_host_genus_id_weighted_unifrac', 'compare_categories_field_host_name_bray_curtis', 'compare_categories_field_host_name_unweighted_unifrac', 'compare_categories_field_host_name_weighted_unifrac', 'compare_categories_reef_name.x_bray_curtis', 'compare_categories_reef_name.x_unweighted_unifrac', 'compare_categories_reef_name.x_weighted_unifrac', 'distance_boxplots_all_within_bray_curtis', 'distan

OSError: [Errno 20] Not a directory: '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/lizard_subset_subset_mucus.biom'

#Compare amongst coral genera

In [45]:
#Take subset of coral genera (field_host_genus_id = Pocillopora, Stylophora, Fungia, Leptastrea, Psammocora, Physogyra, Diploastrea, Caulastrea, Echinopora, Favites, Hydnophora, Lobophyllia, Symphyllia, Turbinaria, Porites, Pachyseris, Galaxea, Acropora, Isopora, Montipora, Pavona, Heliopora, Lobophytum, Millepora, Palythoa)
lizard_clade_otu_table_dir = join(output_dir,'lizard_subset','clade')
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Stylophora.biom -m $mapping -s 'field_host_genus_id:Stylophora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Fungia.biom -m $mapping -s 'field_host_genus_id:Fungia'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Leptastrea.biom -m $mapping -s 'field_host_genus_id:Leptastrea'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Psammocora.biom -m $mapping -s 'field_host_genus_id:Psammocora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Physogyra.biom -m $mapping -s 'field_host_genus_id:Physogyra'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Diploastrea.biom -m $mapping -s 'field_host_genus_id:Diploastrea'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Caulastrea.biom -m $mapping -s 'field_host_genus_id:Caulastrea'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Echinopora.biom -m $mapping -s 'field_host_genus_id:Echinopora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Favites.biom -m $mapping -s 'field_host_genus_id:Favites'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Hydnophora.biom -m $mapping -s 'field_host_genus_id:Hydnophora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Lobophyllia.biom -m $mapping -s 'field_host_genus_id:Lobophyllia'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Symphyllia.biom -m $mapping -s 'field_host_genus_id:Symphyllia'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Turbinaria.biom -m $mapping -s 'field_host_genus_id:Turbinaria'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Porites.biom -m $mapping -s 'field_host_genus_id:Porites'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Pachyseris.biom -m $mapping -s 'field_host_genus_id:Pachyseris'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Galaxea.biom -m $mapping -s 'field_host_genus_id:Galaxea'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Acropora.biom -m $mapping -s 'field_host_genus_id:Acropora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Isopora.biom -m $mapping -s 'field_host_genus_id:Isopora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Montipora.biom -m $mapping -s 'field_host_genus_id:Montipora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Pavona.biom -m $mapping -s 'field_host_genus_id:Pavona'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Heliopora.biom -m $mapping -s 'field_host_genus_id:Heliopora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Lobophytum.biom -m $mapping -s 'field_host_genus_id:Lobophytum'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Millepora.biom -m $mapping -s 'field_host_genus_id:Millepora'
!filter_samples_from_otu_table.py -i $lizard_otu_table -o $lizard_clade_otu_table_dir/lizard_subset_subset_Palythoa.biom -m $mapping -s 'field_host_genus_id:Palythoa'

In [4]:
#Run beta diversity on rarified (1000) subset of each sample type
##THIS WAS RUNNING LAST: Previously, Heliopora and Hydnophora did not receive unifrac distance matrix (_dm.txt) files
#Some genera had too few samples (I believe) to calculate distance matrices.
#Error in make_emperor.py: Due to the variation explained, Emperor could not plot at least 3 axes, check the input files to ensure that the percent explained is greater than 0.01 in at least three axes.

lizard_clade_otu_table_dir = join(output_dir,'lizard_subset','clade')
depth = 1000

for c in listdir(lizard_clade_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(lizard_clade_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(lizard_clade_otu_table_dir,c)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/clade/bdiv_lizard_subset_subset_Acropora.biom'

If you need help with QIIME, see:
http://help.qiime.org
Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/clade/bdiv_lizard_subset_subset_Caulastrea.biom'

If you need help with QIIME, see:
http://help.qiime.org
Error in beta_diversity_through_plots.py: option -i: not a regular file (can't be a directory!): '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_

In [ ]:
#Check for differences among mapping categories within a given clade
lizard_clade_otu_table_dir = join(output_dir,'lizard_subset','clade')

n_permutations = 1000
categories = 'expedition_number,field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id'
for c in listdir(lizard_clade_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(lizard_clade_otu_table_dir,'%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(lizard_clade_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

['bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'lizard_subset_subset_Acropora_even1000.biom', 'log_20160427184217.txt', 'unweighted_unifrac_dm.txt', 'unweighted_unifrac_emperor_pcoa_plot', 'unweighted_unifrac_pc.txt', 'weighted_unifrac_dm.txt', 'weighted_unifrac_emperor_pcoa_plot', 'weighted_unifrac_pc.txt']
curr_cat: expedition_number
curr_metric: bray_curtis
curr_category: bdiv_lizard_subset_subset_Acropora.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                Df SumsOfSqs MeanSqs F.Model      R2   Pr(>F)
qiime.data$map[[opts$category]]  1    0.8714 0.87145  2.0113 0.07732 0.004995
Residuals                       24   10.3988 0.43328         0.92268         
Total                           25   11.2702                 1.00000         
                 

In [48]:
#Boxplot summaries
lizard_clade_otu_table_dir = join(output_dir,'lizard_subset','clade')
n_permutations = 1000
categories = 'expedition_number,field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id,photosynthetically_active_radiation,visibility,turf_contact_percent,n_macroalgal_contacts,cyanobacteria_percent,cyanobacteria_contact,binary_turf_contact,binary_macroalgal_contact'
for c in listdir(lizard_clade_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(lizard_clade_otu_table_dir,'%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

['bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'compare_categories_expedition_number_bray_curtis', 'compare_categories_expedition_number_unweighted_unifrac', 'compare_categories_expedition_number_weighted_unifrac', 'compare_categories_field_host_clade_sensu_fukami_bray_curtis', 'compare_categories_field_host_clade_sensu_fukami_unweighted_unifrac', 'compare_categories_field_host_clade_sensu_fukami_weighted_unifrac', 'compare_categories_field_host_genus_id_bray_curtis', 'compare_categories_field_host_genus_id_unweighted_unifrac', 'compare_categories_field_host_genus_id_weighted_unifrac', 'compare_categories_field_host_name_bray_curtis', 'compare_categories_field_host_name_unweighted_unifrac', 'compare_categories_field_host_name_weighted_unifrac', 'compare_categories_reef_name.x_bray_curtis', 'compare_categories_reef_name.x_unweighted_unifrac', 'compare_categories_reef_name.x_weighted_unifrac', 'lizard_subset_subset_Acropora_even1000.biom', 'log_201604271842

OSError: [Errno 20] Not a directory: '/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/lizard_subset/clade/lizard_subset_subset_Acropora.biom'